# Tutorial 3 - Shared Variables

*"Because text is not enough"* - Anonymous

- `shared_variables` is a dictionary, that is initialised in Agent (default empty dictionary), and can be referenced by any function of the agent (including Inner Agents and their functions)
- This can be useful for non-text modalitiies (e.g. audio, pdfs, image) and lengthy text modalities, which we do not want to output into `subtasks_completed` directly
- `s_` at the start of the variable names means shared variables
    - For input, it means we take the variable from `shared_variables` instead of LLM generated input
    - For output, it means we store the variable into `shared_variables` instead of storing it in `subtasks_completed`. If `subtasks_completed` output is empty, it will be output as `{'Status': 'Completed'}`
- Example shared variables names: `s_sum`, `s_total`, `s_list_of_words`

### Example Input
```python
# Function takes in increment (LLM generated) and s_total (retrieves from shared variable dict), and outputs to s_total (in shared variable dict)
add = Function(fn_description = "Add <increment: int> to <s_total>", 
              output_format = {"s_total": "Modified total"})

# Define the calculator agent and the shared_variables - Note the naming convention of s_ at the start of the names for shared variables
my_agent = Agent('Calculator', 'Does computations', shared_variables = {'s_total': 0}).assign_functions([add])

output = my_agent.run('Increment total by 1')

print('Shared Variables:', my_agent.shared_variables)
```

### Example Output
`Subtask identified: Add 1 to the total`

`Calling function add_int_to_variable with parameters {'increment': 1}`
> {'Status': 'Completed'}

`Task completed successfully!`

`Shared Variables: {'s_total': 1}`

### Example External Function Accessing Shared Variables (Advanced)
```python
# Use shared_variables as input to your external function to access and modify the shared variables
def generate_quotes(shared_variables, number_of_quotes: int, category: str):
    ''' Generates number_of_quotes quotes about category '''
    # Retrieve from shared variables
    my_quote_list = shared_variables['s_quote_list']
    
    ### Add your function code here ###
    
    # Store back to shared variables
    shared_variables['s_quote_list'] = my_quote_list

generate_quote_fn = Function(output_format = {}, external_fn = generate_quotes)
```

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Approach 1: Direct input and output link to shared variable
- Get shared variable as input from `shared_variables`, and output shared variable directly into `shared_variables`
- Both LLM-based functions and external functions can use this approach
- Used when you want to input and output to a shared pool of variables without letting the LLM know the details, e.g. store a persistent state throughout function calls

In [3]:
# Example 1 - LLM-based Function
# Function takes in increment (LLM generated) and s_total (retrieves from shared variable dict), and outputs to s_total (in shared variable dict)
add = Function(fn_description = "Add <increment: int> to <s_total>", 
              output_format = {"s_total": "Modified total"})

In [4]:
# Example 2 - External Functions
def multiply_count(multiplier: int, s_total: int):
    ''' Multiplies s_total by multiplier '''
    return s_total * multiplier

# Function takes inmultiplier (LLM generated) and s_total (retrieves from shared variable dict), and outputs to s_total (in shared variable dict)
multiply = Function(output_format = {"s_total": "Modified total"}, external_fn = multiply_count)

In [5]:
# Define the calculator agent and the shared_variables - Note the naming convention of s_ at the start of the names for shared variables
# Since our agent is like an OS now, we do not want it to generate any response via LLM so we set default_to_llm to be False
my_agent = Agent('Calculator', 'Does computations', default_to_llm = False, shared_variables = {'s_total': 0}).assign_functions([add, multiply])

In [6]:
# visualise the shared variables
print('Shared Variables:', my_agent.shared_variables)

Shared Variables: {'s_total': 0}


In [7]:
# Visualise the stored functions. Note that the shared variables have been replaced to without <> in function description so as not to confuse the LLM
my_agent.print_functions()

Name: end_task
Description: Use only when task is completed
Input: []
Output: {}

Name: add_to_total
Description: Add <increment: int> to s_total
Input: ['increment']
Output: {'s_total': 'Modified total'}

Name: multiply_count
Description:  Multiplies s_total by <multiplier: int> 
Input: ['multiplier']
Output: {'s_total': 'Modified total'}



In [8]:
# You can visualise what are the shared variables in status()
my_agent.status()

Agent Name: Calculator
Agent Description: Does computations
Available Functions: ['end_task', 'add_to_total', 'multiply_count']
Shared Variables: ['s_total']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [9]:
output = my_agent.run('Increment total by 1')

Subtask identified: Use add_to_total function with an increment of 1
Calling function add_to_total with parameters {'increment': 1}
> {'Status': 'Completed'}

Task completed successfully!



In [10]:
# s_total should be successfully updated here as 1
print('Shared Variables:', my_agent.shared_variables)

Shared Variables: {'s_total': 1}


In [11]:
my_agent.reset()
# When doing agents as OS, it is important to reset past subtasks completed so as not to influence future tasks
output = my_agent.run('Increment total by 2, then multiply by 5')

Subtask identified: Increment total by 2
Calling function add_to_total with parameters {'increment': 2}
> {'Status': 'Completed'}

Subtask identified: Multiply total by 5
Calling function multiply_count with parameters {'multiplier': 5}
> {'Status': 'Completed'}

Task completed successfully!



In [12]:
# s_total should be successfully updated here as (1+2)*5 = 15
my_agent.shared_variables

{'s_total': 15}

In [13]:
my_agent.status()

Agent Name: Calculator
Agent Description: Does computations
Available Functions: ['end_task', 'add_to_total', 'multiply_count']
Shared Variables: ['s_total']
Task: Increment total by 2, then multiply by 5
Subtasks Completed:
Subtask: Increment total by 2
{'Status': 'Completed'}

Subtask: Multiply total by 5
{'Status': 'Completed'}

Is Task Completed: True


# Approach 2: Indirect input and output link to shared_variables (only for External Functions)
- Declare the `shared_variables` as a function input in your external function (Note: do not use `**kwargs`)
- You can proceed to access and write to `shared_variables` as needed
- This enables you to process non-text modalities (e.g. audio, pdfs, image) or process lengthy text without outputting to `subtasks_completed`
- Note: You do not have to initialis the names of your variables in `shared_variables` with an s_, as this will be transparent to the meta agent - you can reference any variable name you want in `shared_variables`

In [14]:
# Use shared_variables as input to your external function to access and modify the shared variables
def generate_quotes(shared_variables, number_of_quotes: int, category: str):
    ''' Generates number_of_quotes quotes about category '''
    # Retrieve from shared variables
    my_quote_list = shared_variables['s_quote_list']
    
    # Generate the quotes
    res = strict_json(system_prompt = f'''Generate {number_of_quotes} sentences about {category}. 
Do them in the format "<Quote> - <Person>", e.g. "The way to get started is to quit talking and begin doing. - Walt Disney"
Ensure your quotes contain only ' within the quote, and are enclosed by " ''',
                      user_prompt = '',
                      output_format = {'Quote List': f'list of {number_of_quotes} quotes, type: List[str]'})
    
    my_quote_list.extend([f'Category: {category}. '+ x for x in res['Quote List']])
    
    # Store back to shared variables
    shared_variables['s_quote_list'] = my_quote_list

generate_quote_fn = Function(output_format = {}, external_fn = generate_quotes)

In [15]:
# Define the calculator agent and the shared_variables - Note the naming convention of s_ at the start of the names for shared variables
my_agent = Agent('Input Parser', 'Parses user input to respective functions', 
                 default_to_llm = False,
                 shared_variables = {'s_quote_list': []}).assign_functions([generate_quote_fn])

In [16]:
output = my_agent.run('Generate three quotes about life')

Subtask identified: Generate three quotes about life
Calling function generate_quotes with parameters {'number_of_quotes': 3, 'category': 'life'}
Attempting to use LLM to fix ['"In the end, it's not the years in your life that count. It's the life in your years." - Abraham Lincoln', '"Life is what happens when you're busy making other plans." - John Lennon', '"The purpose of our lives is to be happy." - Dalai Lama'] as it is not a proper array
Fixed list: ['"In the end, it\'s not the years in your life that count. It\'s the life in your years." - Abraham Lincoln', '"Life is what happens when you\'re busy making other plans." - John Lennon', '"The purpose of our lives is to be happy." - Dalai Lama']


> {'Status': 'Completed'}

Task completed successfully!



In [17]:
# visualise quote list
print('Shared Variables:', my_agent.shared_variables)

Shared Variables: {'s_quote_list': ['Category: life. "In the end, it\'s not the years in your life that count. It\'s the life in your years." - Abraham Lincoln', 'Category: life. "Life is what happens when you\'re busy making other plans." - John Lennon', 'Category: life. "The purpose of our lives is to be happy." - Dalai Lama']}


In [18]:
output = my_agent.run('Generate three quotes about happiness')

Subtask identified: Generate three quotes about happiness
Calling function generate_quotes with parameters {'number_of_quotes': 3, 'category': 'happiness'}
> {'Status': 'Completed'}

Task completed successfully!



In [19]:
# visualise quote list
print('Shared Variables:', my_agent.shared_variables)

Shared Variables: {'s_quote_list': ['Category: life. "In the end, it\'s not the years in your life that count. It\'s the life in your years." - Abraham Lincoln', 'Category: life. "Life is what happens when you\'re busy making other plans." - John Lennon', 'Category: life. "The purpose of our lives is to be happy." - Dalai Lama', 'Category: happiness. Happiness is not something ready made. It comes from your own actions. - Dalai Lama', 'Category: happiness. The only thing that will make you happy is being happy with who you are. - Goldie Hawn', 'Category: happiness. Happiness is a choice. You can choose to be happy. - Valerie Bertinelli']}


In [20]:
output = my_agent.run('Generate five quotes about health')

Subtask identified: Generate five quotes about health
Calling function generate_quotes with parameters {'number_of_quotes': 5, 'category': 'health'}
> {'Status': 'Completed'}

Task completed successfully!



In [21]:
# visualise quote list
print('Shared Variables:', my_agent.shared_variables)

Shared Variables: {'s_quote_list': ['Category: life. "In the end, it\'s not the years in your life that count. It\'s the life in your years." - Abraham Lincoln', 'Category: life. "Life is what happens when you\'re busy making other plans." - John Lennon', 'Category: life. "The purpose of our lives is to be happy." - Dalai Lama', 'Category: happiness. Happiness is not something ready made. It comes from your own actions. - Dalai Lama', 'Category: happiness. The only thing that will make you happy is being happy with who you are. - Goldie Hawn', 'Category: happiness. Happiness is a choice. You can choose to be happy. - Valerie Bertinelli', "Category: health. 'Take care of your body. It's the only place you have to live.' - Jim Rohn", "Category: health. 'The greatest wealth is health.' - Virgil", "Category: health. 'An apple a day keeps the doctor away.' - Proverb", "Category: health. 'Healthy is an outfit that looks different on everybody.' - Unknown", "Category: health. 'The groundwork 